In [1]:
# This script is to translate Feidegger dataset annotation from German to English.
# Feidegger is a multi-modal Corpus of Fashion Images and Descriptions in German.
# git clone https://github.com/zalandoresearch/feidegger

In [2]:
data = []
import csv
with open('feidegger/data/FEIDEGGER_release_1.0.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        assert(len(row) == 2)
        if len(row[1]) > 0:
            data.append(row)
print('Read dataset. Processed ' + str(len(data)))

Read dataset. Processed 43941


In [3]:
# import google translate API
from google.cloud import translate_v3beta1 as translate
client = translate.TranslationServiceClient()
project_id = '1050650839547'
location = 'global'
parent = client.location_path(project_id, location)

# test
response = client.translate_text(
    parent=parent,
    contents=['deutsch'],
    mime_type='text/plain',
    source_language_code='de',
    target_language_code='en')
for translation in response.translations:
    print(translation.translated_text)

German


In [4]:
batches = [[]]
for d in data:
    if len(batches[-1]) < 100:
        batches[-1].append(d)
    else:
        batches.append([d])

In [5]:
from tqdm import tqdm
import time
translated_result = []
for batch in tqdm(batches):
    while True:
        try:
            response = client.translate_text(
                parent=parent,
                contents=[d[1] for d in batch],
                mime_type='text/plain',
                source_language_code='de',
                target_language_code='en')
            break
        except Exception as ex:
            if "RESOURCE_EXHAUSTED Quota exceeded for quota group 'billable_default' and limit 'USER-100s'" in ex.message:
                time.sleep(101)
                #global client, parent
                #client = translate.TranslationServiceClient()
                #parent = client.location_path(project_id, location)
            else:
                raise ex
                
    for translation in response.translations:
        translated_result.append(translation.translated_text)

<>:17: SyntaxWarning: name 'client' is used prior to global declaration
<>:17: SyntaxWarning: name 'parent' is used prior to global declaration
<>:17: SyntaxWarning: name 'client' is used prior to global declaration
<>:17: SyntaxWarning: name 'parent' is used prior to global declaration
<>:17: SyntaxWarning: name 'client' is used prior to global declaration
<>:17: SyntaxWarning: name 'parent' is used prior to global declaration
<ipython-input-5-5a86391cf430>:17: SyntaxWarning: name 'client' is used prior to global declaration
  global client, parent
<ipython-input-5-5a86391cf430>:17: SyntaxWarning: name 'parent' is used prior to global declaration
  global client, parent
100%|██████████| 440/440 [1:43:15<00:00, 17.49s/it]  


In [22]:
with open('feidegger/data/FEIDEGGER_release_1.0.english.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    for row, t in zip(data, translated_result):
        csv_writer.writerow([row[0], t])

In [33]:
from IPython.display import Image
from IPython.core.display import HTML 

i = 21
Image(url= data[i][0])

In [34]:

print(translated_result[i])

It is an elegant dark blue mid-length dress with. It has no sleeves. It has a round neckline at the top.
